#### Data generation

In [1]:
import timesynth as ts
import numpy as np
import matplotlib.pyplot as plt
import json
%matplotlib inline

In [ ]:
time_sampler = ts.TimeSampler(stop_time=20)

In [ ]:
irregular_time_samples = time_sampler.sample_irregular_time(num_points=500, keep_percentage=50)

In [ ]:
irregular_time_samples_diff = np.diff(np.append(0, irregular_time_samples))

In [ ]:
phi = 0.10
sigma = 0.2

In [ ]:
signal = np.zeros(len(irregular_time_samples)+1)
noise_samples = np.random.normal(size=len(irregular_time_samples))
for i in range(len(irregular_time_samples)):
    signal[i+1] = np.power(phi, irregular_time_samples_diff[i])*signal[i] + \
                    sigma*np.sqrt(1 - np.power(phi, 2*irregular_time_samples_diff[i]))*noise_samples[i]

In [ ]:
plt.plot(irregular_time_samples, signal[1:])

In [2]:
#Function for building the dictionary
def create_data_dictionary(phi, sigma, irregular_time_samples, signal):
    data_dict = {}
    data_dict['phi'] = phi
    data_dict['sigma'] = sigma
    data_dict['time_samples'] = list(irregular_time_samples)
    data_dict['signal'] = list(signal)
    return data_dict

In [3]:
def write_file(file_name, data_dict):
    with open('../corr_data/'+file_name+'.json', 'w') as fp:
        json.dump(data_dict, fp)

In [4]:
def read_file(file_name):
    with open('../corr_data/'+file_name+'.json', 'r') as fp:
        data = json.load(fp)
    return data

In [5]:
def generate_data(phi, sigma, num_points):
    time_sampler = ts.TimeSampler(stop_time=20)
    irregular_time_samples = time_sampler.sample_irregular_time(num_points=num_points,
                                                                keep_percentage=50)
    irregular_time_samples_diff = np.diff(np.append(0, irregular_time_samples))
    signal = np.zeros(len(irregular_time_samples)+1)
    noise_samples = np.random.normal(size=len(irregular_time_samples))
    for i in range(len(irregular_time_samples)):
        signal[i+1] = np.power(phi, irregular_time_samples_diff[i])*signal[i] + \
                        sigma*np.sqrt(1 - np.power(phi, 2*irregular_time_samples_diff[i]))*noise_samples[i]
    return irregular_time_samples, signal[1:]

#### Automating the process for dataset generation

In [ ]:
num_samples = 10000
phi_samples = np.random.uniform(size=num_samples)
sigma_samples = np.random.uniform(size=num_samples)
signal_lengths = np.random.randint(low=500, high=1000, size=num_samples)

In [ ]:
for i in range(num_samples):
    sigma = sigma_samples[i]
    phi = phi_samples[i]
    sig_length = signal_lengths[i]
    signal = np.nan
    while np.any(np.isnan(signal)):
        time_samples, signal = generate_data(phi, sigma, 600)
    data_dict = create_data_dictionary(phi, sigma, time_samples, signal)
    file_name = 'series_'+str(i)
    write_file(file_name, data_dict)

#### Building the models for learning phi alone

In [6]:
import timeflow as tflow
import tensorflow as tf

In [7]:
class LSTMLayer():
    """
    This layer implements the LSTM cell.
    """
    def __init__(self, input_dim, hidden_layer_size, input_layer):
        """Initialize LSTMLayer class

        Parameters
        ----------
        input_dim : integer
            Input dimensions
        hidden_layer_size : integer
            Size of the memory in LSTM cell
        input_layer : layers object
            Preceding layers object

        """
        self.input_dim = input_dim
        self.hidden_layer_size = hidden_layer_size
        self.inputs = input_layer.get_outputs()

        # Initializing the weights and biases
        self.Wi = tf.Variable(tf.zeros([self.input_dim, self.hidden_layer_size]))
        self.Ui = tf.Variable(tf.zeros([self.hidden_layer_size, self.hidden_layer_size]))
        self.bi = tf.Variable(tf.zeros([self.hidden_layer_size]))

        self.Wf = tf.Variable(tf.zeros([self.input_dim, self.hidden_layer_size]))
        self.Uf = tf.Variable(tf.zeros([self.hidden_layer_size, self.hidden_layer_size]))
        self.bf = tf.Variable(tf.zeros([self.hidden_layer_size]))

        self.Wog = tf.Variable(tf.zeros([self.input_dim, self.hidden_layer_size]))
        self.Uog = tf.Variable(tf.zeros([self.hidden_layer_size, self.hidden_layer_size]))
        self.bog = tf.Variable(tf.zeros([self.hidden_layer_size]))

        self.Wc = tf.Variable(tf.zeros([self.input_dim, self.hidden_layer_size]))
        self.Uc = tf.Variable(tf.zeros([self.hidden_layer_size, self.hidden_layer_size]))
        self.bc = tf.Variable(tf.zeros([self.hidden_layer_size]))

        self.initial_hidden = tf.zeros([1, self.hidden_layer_size])
        self.initial_hidden= tf.pack([self.initial_hidden, self.initial_hidden])

    def forward_step(self, previous_memory, input_):
        """
        Generates the next forward LSTM operation

        Parameters
        ----------
        previous_memory : list
            List of the previous memory and hidden output tensors
        input_ : tf.tensor
            Input tensor

        Returns
        ----------
        list
            New updated memory and hidden output tensors

        """
        previous_hidden_state, c_prev = tf.unpack(previous_memory)
        # Input gate
        i= tf.sigmoid(
            tf.matmul(input_,self.Wi)+tf.matmul(previous_hidden_state,self.Ui) + self.bi
        )
        # Forget Gate
        f= tf.sigmoid(
            tf.matmul(input_,self.Wf)+tf.matmul(previous_hidden_state,self.Uf) + self.bf
        )
        # Output Gate
        o= tf.sigmoid(
            tf.matmul(input_,self.Wog)+tf.matmul(previous_hidden_state,self.Uog) + self.bog
        )
        # New Memory Cell
        c_= tf.nn.tanh(
            tf.matmul(input_,self.Wc)+tf.matmul(previous_hidden_state,self.Uc) + self.bc
        )
        # Final Memory cell
        c= f*c_prev + i*c_

        # Current Hidden state
        current_hidden_state = o*tf.nn.tanh(c)
        return tf.pack([current_hidden_state,c])

    # Function for getting all hidden state.
    def get_outputs(self):
        """
        Iterates through time/ sequence to get all hidden state

        Returns
        ----------
        tf.Tensor
            Output tensor

        """
        # Getting all hidden state throuh time
        inputs_shape = self.inputs.get_shape()
        if inputs_shape[0] == 1:
            self.inputs = tf.expand_dims(self.inputs[0, :, :], 1)
            all_hidden_states = tf.scan(self.forward_step,
                                        self.inputs,
                                        initializer=self.initial_hidden,
                                        name='states')
            all_hidden_states = all_hidden_states[:, 0, :, :]
        else:
            all_hidden_states = tf.map_fn(self.get_batch_outputs,
                                          self.inputs)
        return all_hidden_states
    
    def get_batch_outputs(self, single_input):
        single_input = tf.expand_dims(single_input, 1)
        all_hidden_states = tf.scan(self.forward_step,
                                    single_input,
                                    initializer=self.initial_hidden,
                                    name='states')
        all_hidden_states = all_hidden_states[:, 0, :, :]
        return all_hidden_states

In [8]:
class RegressionLayer():
    """
    Layer implements the Simple regression.
    """
    def __init__(self, input_size, output_size, input_layer):
            """Initialize RegressionLayer class

            Parameters
            ----------
            input_size : integer
                Input dimensions
            output_size : integer
                Output dimensions
            input_layer : layers object
                Preceding layers object

            """
            self.inputs = input_layer.get_outputs()
            self.input_size = input_size
            self.output_size = output_size
            self.Wo = tf.Variable(tf.truncated_normal([self.input_size, self.output_size], mean=0, stddev=.01))
            self.bo = tf.Variable(tf.truncated_normal([self.output_size], mean=0, stddev=.01))

    def get_output(self, input_):
        """
        Generates the output for a single step

        Parameters
        ----------
        input_ : tf.tensor
            Input tensor

        Returns
        ----------
        tf.tensor
            Output tensor

        """
        output = tf.matmul(input_, self.Wo) + self.bo
        return output

    def get_outputs(self):
        """
        Iterates through all inputs to generate outputs

        Returns
        ----------
        tf.Tensor
            Output tensor

        """
        if len(self.inputs.get_shape()) == 3:
            all_outputs = tf.map_fn(self.get_output, self.inputs)
        else:
            all_outputs = tf.map_fn(self.get_batch_outputs, self.inputs)
        return all_outputs
    
    def get_batch_outputs(self, single_input):
        all_single_outputs = tf.map_fn(self.get_output, single_input)
        return all_single_outputs

In [9]:
class MeanLayer():
    """
    Layer implements the Simple mean.
    """
    def __init__(self, input_layer):
        """Initialize MeanLayer class
        """
        self.inputs = input_layer.get_outputs()

    def get_outputs(self):
        """
        Using all inputs to generate outputs
        """
        output = tf.reduce_mean(self.inputs, reduction_indices=1)
        return output

In [10]:
input_size = 2
hidden_size = 100 
output_size = 1
batch_size = 100

In [11]:
with tf.variable_scope('Input'):
    inputs = tflow.placeholders.prediction.input_batch_placeholder(input_size, batch_size)
with tf.variable_scope('Input_LSTM_Layer'):
    input_lstm_layer = tflow.layers.InputLSTMLayer(inputs, input_size, batch_input=True)
with tf.variable_scope('LSTM_Layer'):
    lstm_layer = LSTMLayer(input_size, hidden_size, input_lstm_layer)

In [12]:
with tf.variable_scope('Mean_Layer'):
    mean_layer = MeanLayer(lstm_layer)

In [13]:
with tf.variable_scope('Reg_Layer'):
    reg_layer = RegressionLayer(hidden_size, output_size, mean_layer)

In [14]:
outputs = reg_layer.get_outputs()[:, 0, :]

In [15]:
y = tf.placeholder(tf.float32, shape=[batch_size, output_size],name='outputs')

In [16]:
with tf.variable_scope('RMSE'):
    rmse = tflow.utils.metrics.RMSE(outputs, y)

In [17]:
tf.scalar_summary("RMSE", rmse)
summary_op = tf.merge_all_summaries()

In [18]:
#Training with Adadelta Optimizer
train_step = tf.train.AdamOptimizer(learning_rate=0.2).minimize(rmse)

In [19]:
# Starting tensorflow session
sess=tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

In [20]:
#Setting up log directory for tensorboard
logs_path = 'tmp/corr/1'
writer = tf.train.SummaryWriter(logs_path, graph=tf.get_default_graph())

In [21]:
def build_batch_input(val, batch_size):
    outputs = np.zeros(batch_size)
    counter = 0
    for k in range(val*batch_size, (val+1)*batch_size):
        data_dict = read_file('series_'+str(k))
        time_samples = data_dict['time_samples']
        samples = data_dict['signal']
        outputs[counter] = data_dict['phi']
        if counter == 0:
            input_vector = np.transpose(np.concatenate(
                                    (np.array(time_samples, ndmin=2),
                                     np.array(samples, ndmin=2)), axis=0))
        elif counter == 1:
            value_vector = np.transpose(np.concatenate(
                                    (np.array(time_samples, ndmin=2),
                                     np.array(samples, ndmin=2)), axis=0))
            input_vector = np.stack((input_vector, value_vector))
        else:
            value_vector = np.transpose(np.concatenate(
                                    (np.array(time_samples, ndmin=2),
                                     np.array(samples, ndmin=2)), axis=0))
            input_vector = np.append(input_vector, np.expand_dims(value_vector, 0), axis=0)
        counter += 1
    return input_vector, np.reshape(outputs, (batch_size, 1))

In [22]:
def build_random_batch_input(batch_size):
    outputs = np.zeros(batch_size)
    counter = 0
    random_files = np.random.choice(10000, size=batch_size, replace=False)
    for k in range(batch_size):
        data_dict = read_file('series_'+str(random_files[k]))
        time_samples = data_dict['time_samples']
        samples = data_dict['signal']
        outputs[counter] = data_dict['phi']
        if counter == 0:
            input_vector = np.transpose(np.concatenate(
                                    (np.array(time_samples, ndmin=2),
                                     np.array(samples, ndmin=2)), axis=0))
        elif counter == 1:
            value_vector = np.transpose(np.concatenate(
                                    (np.array(time_samples, ndmin=2),
                                     np.array(samples, ndmin=2)), axis=0))
            input_vector = np.stack((input_vector, value_vector))
        else:
            value_vector = np.transpose(np.concatenate(
                                    (np.array(time_samples, ndmin=2),
                                     np.array(samples, ndmin=2)), axis=0))
            input_vector = np.append(input_vector, np.expand_dims(value_vector, 0), axis=0)
        counter += 1
    return input_vector, np.reshape(outputs, (batch_size, 1))

In [ ]:
for k in range(50):
    input_vector, out_vector = build_random_batch_input(batch_size)
    _, summary = sess.run([train_step, summary_op],
                         feed_dict={inputs:input_vector,
                                    y:out_vector})
    writer.add_summary(summary, k)

In [ ]:
print "Batch "+str(i+1)+" Completed with RMSE: "+str(score)

In [ ]:
data_dict = read_file('series_'+str(0))
time_samples = data_dict['time_samples']
samples = data_dict['signal']

In [ ]:
def build_test_batch_input(file_num, batch_size):
    data_dict = read_file('series_'+str(file_num))
    time_samples = data_dict['time_samples']
    samples = data_dict['signal']
    outputs = data_dict['phi']
    for k in range(batch_size):
        if k == 0:
            input_vector = np.transpose(np.concatenate(
                                    (np.array(time_samples, ndmin=2),
                                     np.array(samples, ndmin=2)), axis=0))
        elif k == 1:
            value_vector = np.transpose(np.concatenate(
                                    (np.array(time_samples, ndmin=2),
                                     np.array(samples, ndmin=2)), axis=0))
            input_vector = np.stack((input_vector, value_vector))
        else:
            value_vector = np.transpose(np.concatenate(
                                    (np.array(time_samples, ndmin=2),
                                     np.array(samples, ndmin=2)), axis=0))
            input_vector = np.append(input_vector, np.expand_dims(value_vector, 0), axis=0)
    return input_vector, outputs

In [ ]:
in_test, value = build_batch_input(1, batch_size)

In [ ]:
out = sess.run(outputs, feed_dict={inputs:in_test})

In [ ]:
value-out

In [ ]:
out